# 22_3 Regression - Nextbike

Focus on Trips starting at stations

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import shapely
import hashlib
import holidays
from shapely import Point, wkt
import scipy.stats
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import shapiro, anderson, norm, expon, gamma
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


## Load and prepare data

In [2]:
data_month_double = '07'

In [3]:
# read final data
data = pd.read_csv("../../../20_Final_Data.csv")

/tmp/ipykernel_1576496/974816303.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../../../20_Final_Data_NEW_TEST.csv")


In [4]:
tier = pd.read_parquet("../Data/12_tier_part2.parquet")
next = pd.read_parquet("../Data/15_nextbike_part2.parquet")

In [5]:
tier['trip_duration_seconds'] = tier['trip_duration'].dt.total_seconds()
next['trip_duration_seconds'] = next['trip_duration'].dt.total_seconds()

In [6]:
# transform geometry
data['station_point'] = data['station_point'].apply(wkt.loads)

In [7]:
data['buffer_zone'] = shapely.wkt.loads(data['buffer_zone'])

In [8]:
data = gpd.GeoDataFrame(data, geometry='station_point')

In [9]:
data = data.loc[:, ~data.columns.str.contains('Unnamed')]

In [10]:
#data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'], inplace=True)

In [11]:
data['actual_arrival_time'] = pd.to_datetime(data['actual_arrival_time'])
data['actual_departure_time'] = pd.to_datetime(data['actual_departure_time'])
data['scheduled_arrival_time'] = pd.to_datetime(data['scheduled_arrival_time'])
data['scheduled_departure_time'] = pd.to_datetime(data['scheduled_departure_time'])

In [12]:
data = gpd.GeoDataFrame(data, geometry='buffer_zone')

In [13]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2564861 entries, 0 to 2564860
Data columns (total 56 columns):
 #   Column                               Dtype         
---  ------                               -----         
 0   route_id                             int64         
 1   agency_id                            int64         
 2   route_short_name                     object        
 3   route_type                           int64         
 4   route_type_name                      object        
 5   agency_name                          object        
 6   service_id                           int64         
 7   trip_id                              object        
 8   trip_headsign                        object        
 9   direction_id                         int64         
 10  shape_id                             int64         
 11  stop_id                              int64         
 12  actual_arrival_time                  datetime64[ns]
 13  actual_departure_ti

In [14]:
data['date'] = pd.to_datetime(data['date'])

In [15]:
data['arrival_delay'] = pd.to_timedelta(data['arrival_delay'])
data['departure_delay'] = pd.to_timedelta(data['departure_delay'])

In [16]:
#NEW FEATURES
data['delay_category'] = data['arrival_delay'].apply(lambda x: 1 if x > pd.Timedelta(0) else 0 if x == pd.Timedelta(0) else -1)
data['cancelled_trip'] = np.where((data['scheduled_arrival_time'].notna() & data['actual_arrival_time'].isna() & data['scheduled_departure_time'].notna() & data['actual_departure_time'].isna()), 1, 0)

In [17]:
data['arrival_delay_float'] = data['arrival_delay'].astype(str)
data['arrival_delay_float'] = data['arrival_delay_float'].str.split(' ', n=2).str[-1]
data['arrival_delay_float'] = data['arrival_delay_float'].astype(str)
data['arrival_delay_float'] = pd.to_timedelta(data['arrival_delay_float'])
data['arrival_delay_float'] = pd.to_timedelta(data['arrival_delay_float']).dt.total_seconds() / 60

In [18]:
data['departure_delay_float'] = data['departure_delay'].astype(str)
data['departure_delay_float'] = data['departure_delay_float'].str.split(' ', n=2).str[-1]
data['departure_delay_float'] = data['departure_delay_float'].astype(str)
data['departure_delay_float'] = pd.to_timedelta(data['departure_delay_float'])
data['departure_delay_float'] = pd.to_timedelta(data['departure_delay_float']).dt.total_seconds() / 60

In [19]:
data['delay'] = data['departure_delay'].apply(lambda x: 1 if x > pd.Timedelta(0) else 0)
data['no_delay'] = data['departure_delay'].apply(lambda x: 1 if x == pd.Timedelta(0) else 0)
data['nan_delay'] = data['departure_delay'].apply(lambda x: 1 if pd.isna(x) else 0)

In [20]:
data['delay_span_int'] = 0

for index in data.index:
    if data['delay_span'][index] == '0':
        data['delay_span_int'][index] = 0
    if data['delay_span'][index] == '0-5':
        data['delay_span_int'][index] = 1
    if data['delay_span'][index] == '5-10':
        data['delay_span_int'][index] = 2
    if data['delay_span'][index] == '10-15':
        data['delay_span_int'][index] = 3
    if data['delay_span'][index] == '15-20':
        data['delay_span_int'][index] = 4
    if data['delay_span'][index] == '20-30':
        data['delay_span_int'][index] = 5
    if data['delay_span'][index] == '30-45':
        data['delay_span_int'][index] = 6
    if data['delay_span'][index] == '45-60':
        data['delay_span_int'][index] = 7
 
    

/tmp/ipykernel_1576496/1846020972.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['delay_span_int'][index] = 0
/tmp/ipykernel_1576496/1846020972.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [21]:
# new colum that sets trips that not happend to -1
data['arrival_delay_new'] = data['arrival_delay'].fillna(-1)
data['arrival_delay_float_new'] = data['arrival_delay_float'].fillna(-1)
# new colum that sets trips that not happend to -1
data['departure_delay_new'] = data['departure_delay'].fillna(-1)
data['departure_delay_float_new'] = data['departure_delay_float'].fillna(-1)

In [22]:
data['current_precipitation_volume'].fillna(0, inplace=True)

/tmp/ipykernel_1576496/969628070.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['current_precipitation_volume'].fillna(0, inplace=True)


In [23]:
delay = data[data['departure_delay'] > pd.Timedelta(0)]


In [24]:
no_delay = data[data['departure_delay'] == pd.Timedelta(0)]

In [25]:
nan_delay = data[data['scheduled_arrival_time'].notna() & data['actual_arrival_time'].isna() & data['scheduled_departure_time'].notna() & data['actual_departure_time'].isna()]
#nan_delay = data[data['arrival_delay'].isna()]
nan_delay

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,direction_id,...,delay_category,cancelled_trip,delay,no_delay,nan_delay,delay_span_int,arrival_delay_new,arrival_delay_float_new,departure_delay_new,departure_delay_float_new
41,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,255-551-008-2071.2.22:033200-34-157_C0741D60-B...,Bonn Hbf,1,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
42,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
43,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
44,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
45,551,8,551,3,Bus,RSVG Rhein-Sieg-Verkehrsgesellschaft mbH,125,159-551-008-687.2.36:044000-33-157_FAA787CB-18...,Troisdorf Bf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564854,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
2564856,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
2564857,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0
2564858,689,6,N9,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,6890005-689-006-687.2.34:263500-53-1_2AFE8F0F-...,Bonn Hbf,0,...,-1,1,0,0,1,0,-1,-1.0,-1,-1.0


### Definition of functions

In [26]:
def calc_stats(data, column_name):

    #generated by Copilot

    sum = data[column_name].sum()

#Central Tendency

    rows = len(data.index)
    # Calculate mean (average)
    mean = data[column_name].mean()
    # Calculate median (middle value)
    median = data[column_name].median()
    # Calculate mode (most frequent value)
    mode = data[column_name].mode()
    #calc maximum
    max = data[column_name].max()
    #calc minimum
    min = data[column_name].min()

    # Calculate range
    data_range = data[column_name].max() - data[column_name].min()
    # Calculate coefficient of variation of range
    cvr = data_range / mean
    # Calculate coefficient of variation of mean
    cvm = mean / mean
    # Calculate coefficient of variation of median
    cvmed = median / mean
    # Calculate coefficient of variation of mode
    cvmode = mode / mean
    # Calculate coefficient of variation of standard deviation
   

# Variability
# Measure of how spread out the values in a data set are
    
    # Calculate standard deviation (average amount of variability in a dataset; tells how far each score lies from the mean; the larger the standard deviation, the more variable the data)
    std_dev = data[column_name].std()
    # Calculate variance (average squared deviation from the mean; square of standard deviation; units of varriance are much larger than those of a typical value in the dataset)
    variance = data[column_name].var()
    # Calculate coefficient of variation
    cv = std_dev / mean # standard deviation divided by the mean; measures the relative variability of a dataset; allows comparison of variability of datasets with different units of measurement
    # Calculate 25th percentile
    percentile_25 = data[column_name].quantile(0.25)
    # Calculate 75th percentile
    percentile_75 = data[column_name].quantile(0.75)
    # Calculate interquartile range
    iqr = percentile_75 - percentile_25 # difference between the 75th and 25th percentiles; measures the spread of the middle 50% of values in a dataset
    # Calculate coefficient of quartile deviation
    qd = iqr / (percentile_75 + percentile_25)
    # Calculate coefficient of variation of coefficient of variation
    cvcv = std_dev / mean
    # Calculate coefficient of variation of coefficient of quartile deviation
    cvqd = iqr / (percentile_75 + percentile_25)
    # Calculate coefficient of variation of range

#Measure of Shape

    # Calculate skewness - measure of how much the probability distribution of a random variable deviates from the normal distribution
    skewness = data[column_name].skew()
    
    # Calculate kurtosis
    kurtosis = data[column_name].kurtosis()

    # Calculate coefficient of skewness
    cs = skewness / std_dev
    # Calculate coefficient of kurtosis
    ck = kurtosis / std_dev
    # Calculate coefficient of variation of skewness
    cvs = skewness / std_dev
    # Calculate coefficient of variation of kurtosis
    cvk = kurtosis / std_dev
    # Calculate coefficient of variation of coefficient of skewness
    cvcs = skewness / std_dev
    # Calculate coefficient of variation of coefficient of kurtosis
    cvck = kurtosis / std_dev
    
    
    
    #return "column: " + str(column_name), "mean: " + str(mean), "median: " + str(median), "mode: " + str(mode), "std_dev: " + str(std_dev), "variance: " + str(variance), "skewness: "+ str(skewness), "kurtosis: " + str(kurtosis), "percentile_25: "+ str(percentile_25), "percentile_75: "+ str(percentile_75), "iqr: " + str(iqr), "range: "+ str(data_range), "cv"+ str(cv), "qd: "+str(qd), "cs: "+ str(cs), "ck: " + str(ck), "csv: "+ str(cvs), "cvk: "+ str(cvk), "cvcs: "+ str(cvcs), "cvck: " +str(cvck), "cvcv: "+ str(cvcv), "cvqd: "+ str (cvqd), "cvr: "+ str(cvr), "cvm: " + str(cvm), "cvmed: " + str(cvmed), "cvmode: "+ str(cvmode)

    return pd.DataFrame({'column': [column_name],
                         'number of rows': [rows],                       
                         'sum': [sum],
                         'mean': [mean],
                         'median': [median],
                         'mode': [mode],
                         'max': [max],
                         'min': [min],
                         'std_dev': [std_dev],
                         'variance': [variance],
                         'skewness': [skewness],
                         'kurtosis': [kurtosis],
                         'percentile_25': [percentile_25],
                         'percentile_75': [percentile_75],
                         'iqr': [iqr],
                         'range': [data_range],
                         'cv': [cv],
                         'qd': [qd],
                         'cs': [cs],
                         'ck': [ck],
                         'cvs': [cvs],
                         'cvk': [cvk],
                         'cvcs': [cvcs],
                         'cvck': [cvck],
                         'cvcv': [cvcv],
                         'cvqd': [cvqd],
                         'cvr': [cvr],
                         'cvm': [cvm],
                         'cvmed': [cvmed],
                         'cvmode': [cvmode]}).T

In [27]:
# create a shorter dataframe with most interesting columns - for better overview
def show_short_df(dataframe):
    delay_short = dataframe[['route_id', 'trip_id', 'stop_id', 'stop_name', 'scheduled_arrival_time', 'actual_arrival_time', 'scheduled_departure_time', 'actual_departure_time', 'arrival_delay', 'departure_delay', 'tier_trips_count', 'tier_trips_end_at_station_count', 'nextbike_trips_count', 'nextbike_trips_end_at_station_count', 'buffer_zone', 'arrival_delay_float', 'departure_delay_float']]
    return delay_short

In [28]:
# transform geometry
point_start = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(tier["start_location"], crs=4326))
point_end = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(tier["end_location"], crs=4326))
# drop geometry columns in wrong format
tier = tier.drop(columns=["start_location", "end_location"])
# replace geometry columns with correct format
tier['start_location'] = point_start
tier['end_location'] = point_end

In [29]:
# get the data from the tier / nextbike dataset
def check_micromobility_datasets(type, data):
    
    if type == 'tier':

        return_data = tier.copy()
        return_data = return_data.iloc[0:0] # empty dataframe

        for item in data['tier_trips_id'].str.split(' ').explode().dropna():   # iterate over all tier ids in the dataframe
            
            id = tier[tier['tier_trips_id'] == int(float(item))]   # get the rows with the specific id
            return_data = pd.concat([return_data, id])  # add the rows to the return dataframe

            return_data.drop_duplicates(inplace=True)
            
    if type == 'next':

        return_data = next.copy()
        return_data = return_data.iloc[0:0] # empty dataframe

        for item in data['nextbike_trips_id'].str.split(' ').explode().dropna():   # iterate over all nextbike ids in the dataframe
            
            id = next[next['nextbike_trips_id'] == int(float(item))]  # get the rows with the specific id
            return_data = pd.concat([return_data, id]) # add the rows to the return dataframe

            return_data.drop_duplicates(inplace=True)

    return return_data

In [30]:
def get_station_to_station(data, mode):
    
    if mode == 'tier':

        data = data[data['tier_trips_start_at_station_id'].notna() & data['tier_trips_end_at_station_id'].notna()]
        data = data[data['tier_trips_start_at_station_id'] != data['tier_trips_end_at_station_id']]
        data = data[data['tier_trips_start_at_station_id'] != 0]
        data = data[data['tier_trips_end_at_station_id'] != 0]
    t = check_micromobility_datasets('tier', data)
    n = check_micromobility_datasets('next', data)

    t = t[t['end_stop_id'].notna() & t['start_stop_id'].notna()]
    n = n[n['end_stop_id'].notna() & n['start_stop_id'].notna()]
    
    return t, n

In [31]:
# map the start and end locations of the trips on a map and connect them with a line

def map_buffer_zones(dataset):
    
    micromobility_data = check_micromobility_datasets('tier', dataset)
    
    # Create a folium map object
    m = folium.Map(location=[50.73743, 7.09821], zoom_start=12)

    # Iterate over each row in the micromobility data
    for index, row in micromobility_data.iterrows():
        # Get the start and end locations
        start_location = row['start_location']
        end_location = row['end_location']
        
        # Get the start and end stop names
        start_stop_name = row['start_stop_name']
        end_stop_name = row['end_stop_name']
        # Add markers for the start and end locations
        folium.Marker(location=[start_location.y, start_location.x], popup=start_stop_name).add_to(m)
        folium.Marker(location=[end_location.y, end_location.x], popup=end_stop_name).add_to(m)
        # Create a line connecting the start and end locations
        folium.PolyLine(locations=[[start_location.y, start_location.x], [end_location.y, end_location.x]], color='blue').add_to(m)
        
    # Display the map
    return m

In [32]:
def get_stats(data, var):

    if var == 'all':
        var = data.columns

    df = pd.DataFrame()

    for i in var:
        df = pd.concat([df, calc_stats(data, i)], axis=1)

    df.columns = df.iloc[0]
    df = df[1:]
    
    return df

In [33]:
def get_micro_stats(data, operator):
    
    if operator == 'tier':
        dataframe = get_stats(check_micromobility_datasets(operator, data), ['trip_duration_seconds', 'trip_distance_kilometers'])
    elif operator == 'next':
        dataframe = get_stats(check_micromobility_datasets(operator, data), ['trip_duration_seconds', 'trip_distance_kilometers'])
    
    return dataframe

In [34]:
def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i, y[i], y[i], ha = 'center')
                 #Bbox = dict(facecolor = 'red', alpha =.8))

route_id', 'agency_id',
       'route_short_name', 'route_type', 'route_type_name', 'agency_name',
       'service_id', 'trip_id', 'trip_headsign', 'direction_id', 'shape_id',
       'stop_id', 'actual_arrival_time', 'actual_departure_time',
       'vrs_timestamp', 'stop_sequence', 'stop_headsign', 'stop_name',
       'station_point', 'buffer_zone', 'scheduled_arrival_time',
       'scheduled_departure_time', 'arrival_delay', 'departure_delay',
       'service', 'date', 'weekday', 'transfer_stop', 'tier_trips_count',
       'tier_trips_id', 'nextbike_trips_count', 'nextbike_trips_id',
       'current_time', 'current_temp', 'current_feels_like',
       'current_cloudiness', 'current_visibility', 'current_wind_speed',
       'current_description', 'current_precipitation_volume', 'holiday',
       'holiday_NRW', 'holiday_name', 'holiday_name_NRW'

In [35]:
data.columns

Index(['route_id', 'agency_id', 'route_short_name', 'route_type',
       'route_type_name', 'agency_name', 'service_id', 'trip_id',
       'trip_headsign', 'direction_id', 'shape_id', 'stop_id',
       'actual_arrival_time', 'actual_departure_time', 'vrs_timestamp',
       'stop_sequence', 'stop_headsign', 'stop_name', 'scheduled_arrival_time',
       'scheduled_departure_time', 'arrival_delay', 'departure_delay',
       'service', 'date', 'weekday', 'transfer_stop', 'tier_trips_count',
       'tier_trips_id', 'tier_trips_end_at_station_count',
       'tier_trips_end_at_station_id', 'station_point', 'buffer_zone',
       'nextbike_trips_count', 'nextbike_trips_id',
       'nextbike_trips_end_at_station_count',
       'nextbike_trips_end_at_station_id', 'current_time', 'current_temp',
       'current_feels_like', 'current_cloudiness', 'current_visibility',
       'current_wind_speed', 'current_description',
       'current_precipitation_volume', 'arrival_delay_float',
       'departure_

## Def Features

In [36]:
y_string = 'nextbike_trips_count'

In [37]:
# create dummy variables for categories
delay_span_dummy = data[['delay_span']]
delay_span_dummy = pd.get_dummies(delay_span_dummy, columns=['delay_span'], drop_first=True)


weekday_dummy = data[['weekday']]
weekday_dummy = pd.get_dummies(weekday_dummy, columns=['weekday'], drop_first=True)


time_span_dummy = data[['time_span']]
time_span_dummy = pd.get_dummies(time_span_dummy, columns=['time_span'], drop_first=True)

data_slim = data.drop(columns=['route_id', 'agency_id', 'route_short_name',
       'route_type_name', 'agency_name', 'service_id', 'trip_id',
       'trip_headsign', 'direction_id', 'shape_id',
       'actual_arrival_time', 'actual_departure_time', 'vrs_timestamp',
       'stop_sequence', 'stop_headsign', 'stop_name', 'scheduled_arrival_time',
       'scheduled_departure_time', 'arrival_delay', 'date', 'tier_trips_id', 'tier_trips_end_at_station_count',
       'tier_trips_end_at_station_id', 'station_point', 'buffer_zone',
       'tier_trips_count', 'nextbike_trips_id',
       'nextbike_trips_end_at_station_count', 'departure_delay', 'actual_arrival_time_float', 'time_span_int', 'delay_category', 'departure_delay_new',
       'nextbike_trips_end_at_station_id', 'current_time', 'current_feels_like', 'current_cloudiness', 'current_visibility',
       'current_wind_speed', 'current_description', 'arrival_delay_float',  'actual_departure_time_float',  'time_span_arrival', 'time_span_int_arrival', 'delay_span_int', 'arrival_delay_new', 'arrival_delay_float_new', 'arrival_delay_span'])

data_dummy = pd.get_dummies(data_slim, columns=['time_span', 'weekday', 'delay_span'], drop_first=True)

In [38]:
data_slim.columns

Index(['route_type', 'stop_id', 'service', 'weekday', 'transfer_stop',
       'nextbike_trips_count', 'current_temp', 'current_precipitation_volume',
       'departure_delay_float', 'delay_span', 'weather_int', 'weekend',
       'time_span', 'cancelled_trip', 'delay', 'no_delay', 'nan_delay',
       'departure_delay_float_new'],
      dtype='object')

In [39]:
data_dummy.columns

Index(['route_type', 'stop_id', 'service', 'transfer_stop',
       'nextbike_trips_count', 'current_temp', 'current_precipitation_volume',
       'departure_delay_float', 'weather_int', 'weekend', 'cancelled_trip',
       'delay', 'no_delay', 'nan_delay', 'departure_delay_float_new',
       'time_span_afternoon', 'time_span_early morning', 'time_span_evening',
       'time_span_evening rush', 'time_span_lunch', 'time_span_morning rush',
       'time_span_until midday', 'weekday_Monday', 'weekday_Saturday',
       'weekday_Sunday', 'weekday_Thursday', 'weekday_Tuesday',
       'weekday_Wednesday', 'delay_span_0-5', 'delay_span_10-15',
       'delay_span_15-20', 'delay_span_20-30', 'delay_span_30-45',
       'delay_span_45-60', 'delay_span_5-10'],
      dtype='object')

# Test for Model

In [51]:
# Varianz und Mittelwert der abhängigen Variable
# Var > Mean -> Poisson nicht gut (bei großer Abweichung)
mean = data[y_string].mean()
print(mean)
variance = data[y_string].var()
print(variance)
dispersion_ratio = variance / mean
print(dispersion_ratio)

0.04864669079532965
0.0734255495825432
1.5093637076253594


In [56]:
corr_matrix = data_dummy.corr()
corr_matrix

,route_type,stop_id,service,transfer_stop,nextbike_trips_count,current_temp,current_precipitation_volume,departure_delay_float,weather_int,weekend,...,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,delay_span_0-5,delay_span_10-15,delay_span_15-20,delay_span_20-30,delay_span_30-45,delay_span_45-60,delay_span_5-10
route_type,1.000000,0.060355,0.016856,-0.065857,-0.017866,0.027113,-0.001331,0.026825,-0.003618,-0.008933,...,0.004911,0.002984,0.004350,0.059182,0.005625,0.008414,0.008273,0.007988,0.005451,0.008172
stop_id,0.060355,1.000000,-0.001685,-0.015772,-0.059338,0.006173,-0.000459,-0.000539,-0.001419,-0.001467,...,0.001104,0.000740,0.001130,0.019224,0.030173,-0.004215,0.014147,-0.006972,-0.009670,-0.006247
service,0.016856,-0.001685,1.000000,-0.007632,0.008975,0.050747,0.001262,-0.020214,-0.000763,-0.095210,...,0.027363,0.027405,0.027388,0.012627,-0.001942,-0.002099,-0.007796,-0.034543,-0.026784,-0.000539
transfer_stop,-0.065857,-0.015772,-0.007632,1.000000,0.036869,-0.005364,-0.000140,-0.001756,0.000614,0.006757,...,-0.001854,-0.001280,-0.001870,-0.043125,-0.005749,0.002988,0.011454,0.014965,-0.004798,-0.002561
nextbike_trips_count,-0.017866,-0.059338,0.008975,0.036869,1.000000,0.037952,-0.009247,-0.009838,-0.018254,-0.019111,...,0.003624,0.006656,0.008540,-0.031373,-0.003003,-0.003589,-0.003144,-0.003580,0.004147,-0.002867
current_temp,0.027113,0.006173,0.050747,-0.005364,0.037952,1.000000,-0.049263,-0.016466,-0.142011,0.135245,...,-0.126978,0.078613,-0.106187,0.025606,-0.002046,0.000572,-0.001695,-0.021065,-0.014491,0.008926
current_precipitation_volume,-0.001331,-0.000459,0.001262,-0.000140,-0.009247,-0.049263,1.000000,-0.003632,0.685637,0.087984,...,0.027576,-0.066925,-0.042474,-0.006357,-0.000857,-0.000513,-0.000959,-0.002646,-0.002144,-0.000870
departure_delay_float,0.026825,-0.000539,-0.020214,-0.001756,-0.009838,-0.016466,-0.003632,1.000000,-0.005550,0.006572,...,-0.002835,-0.001682,-0.002564,0.044318,0.091897,0.200583,0.245863,0.384454,0.191557,0.058298
weather_int,-0.003618,-0.001419,-0.000763,0.000614,-0.018254,-0.142011,0.685637,-0.005550,1.000000,0.067783,...,0.068347,-0.056565,-0.062573,-0.007977,-0.000623,-0.000165,-0.001100,-0.004030,-0.002951,-0.002118
weekend,-0.008933,-0.001467,-0.095210,0.006757,-0.019111,0.135245,0.087984,0.006572,0.067783,1.000000,...,-0.288432,-0.288875,-0.288693,0.011333,0.006153,0.003714,-0.000079,0.011069,0.006178,-0.007732


In [57]:
# Calculate the correlation matrix
corr_matrix = data_dummy.corr()

# Select the target variable (dependent variable)
target_variable = y_string

# Sort the correlation matrix by the absolute values of the correlation with the target variable
sorted_corr = corr_matrix[target_variable].abs().sort_values(ascending=False)

# Select the top k features with the highest correlation with the target variable
k = 10
selected_features = sorted_corr[1:k+1].index.tolist()

# Print the selected features
print("Selected features:")
print(selected_features)

Selected features:
['stop_id', 'no_delay', 'delay', 'current_temp', 'transfer_stop', 'delay_span_0-5', 'time_span_afternoon', 'weekend', 'weekday_Sunday', 'weather_int']


In [58]:
selected_features = data_dummy[selected_features]

# Linear Regression

### Best Model

In [73]:
X = selected_features
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog)
result = poission_model.fit()
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564850
Model Family:                  Gaussian   Df Model:                           10
Link Function:                 Identity   Scale:                        0.072593
Method:                            IRLS   Log-Likelihood:            -2.7571e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   1.8619e+05
Time:                          18:19:10   Pearson chi2:                 1.86e+05
No. Iterations:                       3   Pseudo R-squ. (CS):            0.01140
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0486      0.000

In [72]:
X = data_dummy.drop(columns=["nextbike_trips_count", "departure_delay_float"])
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog)
result = poission_model.fit()
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564829
Model Family:                  Gaussian   Df Model:                           31
Link Function:                 Identity   Scale:                        0.072431
Method:                            IRLS   Log-Likelihood:            -2.7283e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   1.8577e+05
Time:                          18:19:04   Pearson chi2:                 1.86e+05
No. Iterations:                     100   Pseudo R-squ. (CS):            0.01365
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0485      0.000

### Other

In [59]:
short_data = data[['departure_delay', y_string]].copy()
short_data['departure_delay'] = pd.to_numeric(short_data['departure_delay'])

In [60]:
model = smf.ols(formula='nextbike_trips_count ~ departure_delay', data=short_data).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                      -0.019
Model:                              OLS   Adj. R-squared:                 -0.019
Method:                   Least Squares   F-statistic:                       nan
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):                nan
Time:                          18:08:43   Log-Likelihood:            -3.1426e+05
No. Observations:               2564861   AIC:                         6.285e+05
Df Residuals:                   2564860   BIC:                         6.285e+05
Df Model:                             0                                         
Covariance Type:              nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        5.784

In [61]:
model = smf.ols(formula='nextbike_trips_count ~ departure_delay_float', data=data).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.000
Model:                              OLS   Adj. R-squared:                  0.000
Method:                   Least Squares   F-statistic:                     147.4
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):           6.49e-34
Time:                          18:08:43   Log-Likelihood:            -1.5568e+05
No. Observations:               1522668   AIC:                         3.114e+05
Df Residuals:                   1522666   BIC:                         3.114e+05
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept 

In [62]:
model = smf.ols(formula='nextbike_trips_count ~ departure_delay_new', data=data).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.004
Model:                              OLS   Adj. R-squared:                  0.004
Method:                   Least Squares   F-statistic:                     53.54
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                          18:10:04   Log-Likelihood:            -2.8555e+05
No. Observations:               2564861   AIC:                         5.715e+05
Df Residuals:                   2564681   BIC:                         5.738e+05
Df Model:                           179                                         
Covariance Type:              nonrobust                                         
                                                          coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [63]:
model = smf.ols(formula='nextbike_trips_count ~ departure_delay_float_new', data=data).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.000
Model:                              OLS   Adj. R-squared:                  0.000
Method:                   Least Squares   F-statistic:                     150.6
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):           1.27e-34
Time:                          18:10:06   Log-Likelihood:            -2.9026e+05
No. Observations:               2564861   AIC:                         5.805e+05
Df Residuals:                   2564859   BIC:                         5.805e+05
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
In

In [64]:
model = smf.ols(formula='tier_trips_count ~ delay_span', data=data).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:       tier_trips_count   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     338.6
Date:                Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                        18:10:19   Log-Likelihood:             1.1174e+06
No. Observations:             2564861   AIC:                        -2.235e+06
Df Residuals:                 2564853   BIC:                        -2.235e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0228    

In [65]:
model = smf.ols(formula='nextbike_trips_count ~ cancelled_trip', data=data).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.000
Model:                              OLS   Adj. R-squared:                  0.000
Method:                   Least Squares   F-statistic:                     11.60
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):           0.000658
Time:                          18:10:21   Log-Likelihood:            -2.9033e+05
No. Observations:               2564861   AIC:                         5.807e+05
Df Residuals:                   2564859   BIC:                         5.807e+05
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.049

In [66]:
model = smf.ols(formula='nextbike_trips_count ~ delay', data=data).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.001
Model:                              OLS   Adj. R-squared:                  0.001
Method:                   Least Squares   F-statistic:                     3703.
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                          18:10:23   Log-Likelihood:            -2.8848e+05
No. Observations:               2564861   AIC:                         5.770e+05
Df Residuals:                   2564859   BIC:                         5.770e+05
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0565      0.000

In [67]:
model = smf.ols(formula='nextbike_trips_count ~ no_delay', data=data).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.002
Model:                              OLS   Adj. R-squared:                  0.002
Method:                   Least Squares   F-statistic:                     4539.
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                          18:10:24   Log-Likelihood:            -2.8807e+05
No. Observations:               2564861   AIC:                         5.761e+05
Df Residuals:                   2564859   BIC:                         5.762e+05
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0425      0.000

In [68]:
model = smf.ols(formula='nextbike_trips_count ~ nan_delay', data=data).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.000
Model:                              OLS   Adj. R-squared:                  0.000
Method:                   Least Squares   F-statistic:                     7.424
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):            0.00643
Time:                          18:10:25   Log-Likelihood:            -2.9033e+05
No. Observations:               2564861   AIC:                         5.807e+05
Df Residuals:                   2564859   BIC:                         5.807e+05
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0483      0.000

In [69]:
model = smf.ols(formula='nextbike_trips_count ~ nan_delay', data=data).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.000
Model:                              OLS   Adj. R-squared:                  0.000
Method:                   Least Squares   F-statistic:                     7.424
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):            0.00643
Time:                          18:10:27   Log-Likelihood:            -2.9033e+05
No. Observations:               2564861   AIC:                         5.807e+05
Df Residuals:                   2564859   BIC:                         5.807e+05
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0483      0.000

In [70]:
X = data[['departure_delay_float_new', 'current_temp', 'current_precipitation_volume', 'service', 'delay_span_int', 'time_span_int', 'weekend', 'delay_category', 'cancelled_trip', 'delay', 'no_delay', 'nan_delay']]
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.OLS(y, exog)
result = poission_model.fit()
print(result.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.005
Model:                              OLS   Adj. R-squared:                  0.005
Method:                   Least Squares   F-statistic:                     1136.
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                          18:10:32   Log-Likelihood:            -2.8410e+05
No. Observations:               2564861   AIC:                         5.682e+05
Df Residuals:                   2564849   BIC:                         5.684e+05
Df Model:                            11                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0486      0.000

In [71]:
data_dummy.isna().any()

route_type                      False
stop_id                         False
service                         False
transfer_stop                   False
nextbike_trips_count            False
current_temp                    False
current_precipitation_volume    False
departure_delay_float            True
weather_int                     False
weekend                         False
cancelled_trip                  False
delay                           False
no_delay                        False
nan_delay                       False
departure_delay_float_new       False
time_span_afternoon             False
time_span_early morning         False
time_span_evening               False
time_span_evening rush          False
time_span_lunch                 False
time_span_morning rush          False
time_span_until midday          False
weekday_Monday                  False
weekday_Saturday                False
weekday_Sunday                  False
weekday_Thursday                False
weekday_Tues

# Poisson Regression

### Best Model

In [40]:
agg_station_date_10min = data.groupby(['stop_id', 'stop_name', pd.Grouper(key='scheduled_arrival_time', freq='10Min')]).sum(numeric_only=True)

In [41]:
agg_station_date_10min.isna().any()

route_id                               False
agency_id                              False
route_type                             False
service_id                             False
direction_id                           False
shape_id                               False
vrs_timestamp                          False
stop_sequence                          False
service                                False
transfer_stop                          False
tier_trips_count                       False
tier_trips_end_at_station_count        False
nextbike_trips_count                   False
nextbike_trips_end_at_station_count    False
current_temp                           False
current_feels_like                     False
current_cloudiness                     False
current_visibility                     False
current_wind_speed                     False
current_precipitation_volume           False
arrival_delay_float                    False
departure_delay_float                  False
actual_arr

In [43]:
poisson_model = smf.poisson('nextbike_trips_count ~ departure_delay_float_new', data=agg_station_date_10min).fit()
print(poisson_model.summary())

Optimization terminated successfully.
         Current function value: 0.343798
         Iterations 18
                           Poisson Regression Results                           
Dep. Variable:     nextbike_trips_count   No. Observations:              1290080
Model:                          Poisson   Df Residuals:                  1290078
Method:                             MLE   Df Model:                            1
Date:                  Fri, 13 Sep 2024   Pseudo R-squ.:                  0.2082
Time:                          18:08:29   Log-Likelihood:            -4.4353e+05
converged:                         True   LL-Null:                   -5.6016e+05
Covariance Type:              nonrobust   LLR p-value:                     0.000
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -2.8631      0.003   -843.252   

In [89]:
X = selected_features
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit(method="lbfgs")
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564850
Model Family:                   Poisson   Df Model:                           10
Link Function:                      Log   Scale:                          1.0000
Method:                           lbfgs   Log-Likelihood:            -4.9854e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   7.8253e+05
Time:                          18:20:35   Pearson chi2:                 4.12e+06
No. Iterations:                      14   Pseudo R-squ. (CS):            0.01837
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.2603      0.004

In [84]:
X = data_dummy.drop(columns=[y_string, 'departure_delay_float'])
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit(method="lbfgs")
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564829
Model Family:                   Poisson   Df Model:                           31
Link Function:                      Log   Scale:                          1.0000
Method:                           lbfgs   Log-Likelihood:            -4.9620e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   7.7786e+05
Time:                          18:20:09   Pearson chi2:                 4.11e+06
No. Iterations:                      20   Pseudo R-squ. (CS):            0.02016
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.2846      0.004

### Other

In [74]:
X = data[['departure_delay']]
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564859
Model Family:                   Poisson   Df Model:                            1
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.2231e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   8.3008e+05
Time:                          18:19:13   Pearson chi2:                 3.87e+06
No. Iterations:                       7   Pseudo R-squ. (CS):          4.364e-06
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0232      0.003

In [75]:
X = data[['departure_delay_float_new']]
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564859
Model Family:                   Poisson   Df Model:                            1
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.2193e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   8.2932e+05
Time:                          18:19:16   Pearson chi2:                 3.87e+06
No. Iterations:                       7   Pseudo R-squ. (CS):          0.0002992
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0283      0.003

In [76]:
from sklearn.preprocessing import StandardScaler


X = delay_span_dummy
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564853
Model Family:                   Poisson   Df Model:                            7
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.2013e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   8.2571e+05
Time:                          18:19:21   Pearson chi2:                 3.81e+06
No. Iterations:                       7   Pseudo R-squ. (CS):           0.001705
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0425      0.003

In [77]:
from sklearn.preprocessing import StandardScaler

X = data[['delay_category']]
y = data['tier_trips_count']

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:       tier_trips_count   No. Observations:              2564861
Model:                            GLM   Df Residuals:                  2564859
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -2.5316e+05
Date:                Fri, 13 Sep 2024   Deviance:                   4.1279e+05
Time:                        18:19:24   Pearson chi2:                 3.19e+06
No. Iterations:                     7   Pseudo R-squ. (CS):          0.0001907
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.9308      0.004   -879.756      0.0

In [78]:
from sklearn.preprocessing import StandardScaler


X = time_span_dummy
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564853
Model Family:                   Poisson   Df Model:                            7
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.1982e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   8.2510e+05
Time:                          18:19:29   Pearson chi2:                 3.84e+06
No. Iterations:                       7   Pseudo R-squ. (CS):           0.001944
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0419      0.003

In [79]:
from sklearn.preprocessing import StandardScaler


X = weekday_dummy
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564854
Model Family:                   Poisson   Df Model:                            6
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.2116e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   8.2777e+05
Time:                          18:19:34   Pearson chi2:                 3.85e+06
No. Iterations:                       7   Pseudo R-squ. (CS):          0.0009026
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0334      0.003

In [80]:
from sklearn.preprocessing import StandardScaler

X = data[['cancelled_trip']]
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564859
Model Family:                   Poisson   Df Model:                            1
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.2230e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   8.3007e+05
Time:                          18:19:37   Pearson chi2:                 3.87e+06
No. Iterations:                       7   Pseudo R-squ. (CS):          6.841e-06
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0232      0.003

In [81]:

X = data[['nan_delay']]
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564859
Model Family:                   Poisson   Df Model:                            1
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.2231e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   8.3008e+05
Time:                          18:19:39   Pearson chi2:                 3.87e+06
No. Iterations:                       7   Pseudo R-squ. (CS):          4.364e-06
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0232      0.003

In [82]:
X = data[['no_delay']]
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564859
Model Family:                   Poisson   Df Model:                            1
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.1918e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   8.2382e+05
Time:                          18:19:42   Pearson chi2:                 3.81e+06
No. Iterations:                       7   Pseudo R-squ. (CS):           0.002441
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0465      0.003

In [83]:
X = data[['delay']]
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit()
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564859
Model Family:                   Poisson   Df Model:                            1
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.1939e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   8.2423e+05
Time:                          18:19:45   Pearson chi2:                 3.78e+06
No. Iterations:                       7   Pseudo R-squ. (CS):           0.002279
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0480      0.003

#### Multi

In [85]:
X = data_dummy.drop(columns=[y_string, 'departure_delay_float'])
X

,route_type,stop_id,service,transfer_stop,current_temp,current_precipitation_volume,weather_int,weekend,cancelled_trip,delay,...,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,delay_span_0-5,delay_span_10-15,delay_span_15-20,delay_span_20-30,delay_span_30-45,delay_span_45-60,delay_span_5-10
0,3,687,1,False,18.47,0.00,0,1,0,1,...,False,False,False,False,False,False,False,False,False,False
1,3,1106,1,True,18.47,0.00,0,1,0,1,...,False,False,False,False,False,False,False,False,False,False
2,3,1102,1,False,18.47,0.00,0,1,0,1,...,False,False,False,False,False,False,False,False,False,False
3,3,1115,1,True,18.47,0.00,0,1,0,1,...,False,False,False,False,False,False,False,False,False,False
4,3,1500,1,True,18.47,0.00,0,1,0,1,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564856,3,1204,1,False,17.30,1.34,1,0,1,0,...,False,True,False,False,False,False,False,False,False,False
2564857,3,1140,1,False,17.30,1.34,1,0,1,0,...,False,True,False,False,False,False,False,False,False,False
2564858,3,1141,1,False,17.30,1.34,1,0,1,0,...,False,True,False,False,False,False,False,False,False,False
2564859,3,687,1,False,17.30,1.34,1,0,0,0,...,False,True,False,False,False,False,False,False,False,False


In [86]:
X = time_span_dummy
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit(method="lbfgs")
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564853
Model Family:                   Poisson   Df Model:                            7
Link Function:                      Log   Scale:                          1.0000
Method:                           lbfgs   Log-Likelihood:            -5.1982e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   8.2510e+05
Time:                          18:20:14   Pearson chi2:                 3.84e+06
No. Iterations:                       4   Pseudo R-squ. (CS):           0.001944
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0418      0.003

In [87]:
X = delay_span_dummy
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit(method="lbfgs")
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564853
Model Family:                   Poisson   Df Model:                            7
Link Function:                      Log   Scale:                          1.0000
Method:                           lbfgs   Log-Likelihood:            -5.2013e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   8.2571e+05
Time:                          18:20:20   Pearson chi2:                 3.81e+06
No. Iterations:                       8   Pseudo R-squ. (CS):           0.001705
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0427      0.003

In [88]:
X = weekday_dummy
y = data[y_string]

standizer = StandardScaler()
X = standizer.fit_transform(X)

exog = sm.add_constant(X)
poission_model = sm.GLM(y, exog, family=sm.families.Poisson())
result = poission_model.fit(method="lbfgs")
print(result.summary())

                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564854
Model Family:                   Poisson   Df Model:                            6
Link Function:                      Log   Scale:                          1.0000
Method:                           lbfgs   Log-Likelihood:            -5.2116e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   8.2777e+05
Time:                          18:20:25   Pearson chi2:                 3.85e+06
No. Iterations:                       6   Pseudo R-squ. (CS):          0.0009026
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0333      0.003

# Zero Inflated Poisson Regression

In [90]:
import statsmodels.api as sm

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string], data[['departure_delay_float_new']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.195094
         Iterations: 14
         Function evaluations: 17
         Gradient evaluations: 17
                     ZeroInflatedPoisson Regression Results                     
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:              ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                             MLE   Df Model:                            0
Date:                  Fri, 13 Sep 2024   Pseudo R-squ.:                -0.02464
Time:                          18:21:14   Log-Likelihood:            -5.0039e+05
converged:                         True   LL-Null:                   -4.8835e+05
Covariance Type:              nonrobust   LLR p-value:                       nan
                                coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------

In [91]:

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string], data[['delay_span_int']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.192885
         Iterations: 15
         Function evaluations: 17
         Gradient evaluations: 17
                     ZeroInflatedPoisson Regression Results                     
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:              ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                             MLE   Df Model:                            0
Date:                  Fri, 13 Sep 2024   Pseudo R-squ.:                -0.01304
Time:                          18:21:58   Log-Likelihood:            -4.9472e+05
converged:                         True   LL-Null:                   -4.8835e+05
Covariance Type:              nonrobust   LLR p-value:                       nan
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
inflate_const 

In [92]:

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string], data[['delay_category']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.195057
         Iterations: 13
         Function evaluations: 15
         Gradient evaluations: 15
                     ZeroInflatedPoisson Regression Results                     
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:              ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                             MLE   Df Model:                            0
Date:                  Fri, 13 Sep 2024   Pseudo R-squ.:                -0.02445
Time:                          18:22:40   Log-Likelihood:            -5.0029e+05
converged:                         True   LL-Null:                   -4.8835e+05
Covariance Type:              nonrobust   LLR p-value:                       nan
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
inflate_const 

In [93]:
data_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2564861 entries, 0 to 2564860
Data columns (total 35 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   route_type                    int64  
 1   stop_id                       int64  
 2   service                       int64  
 3   transfer_stop                 bool   
 4   nextbike_trips_count          int64  
 5   current_temp                  float64
 6   current_precipitation_volume  float64
 7   departure_delay_float         float64
 8   weather_int                   int64  
 9   weekend                       int64  
 10  cancelled_trip                int64  
 11  delay                         int64  
 12  no_delay                      int64  
 13  nan_delay                     int64  
 14  departure_delay_float_new     float64
 15  time_span_afternoon           bool   
 16  time_span_early morning       bool   
 17  time_span_evening             bool   
 18  time_span_evening rush

In [94]:
X = data_dummy.drop(columns=[y_string, "departure_delay_float"])
y = data[y_string]

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(y,X.astype(float))

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.184902
         Iterations: 35
         Function evaluations: 41
         Gradient evaluations: 41


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                     ZeroInflatedPoisson Regression Results                     
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:              ZeroInflatedPoisson   Df Residuals:                  2564828
Method:                             MLE   Df Model:                           32
Date:                  Fri, 13 Sep 2024   Pseudo R-squ.:                 0.02888
Time:                          18:25:08   Log-Likelihood:            -4.7425e+05
converged:                        False   LL-Null:                   -4.8835e+05
Covariance Type:              nonrobust   LLR p-value:                     0.000
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
inflate_const                    1.6943      0.004    466.828      0.000       1.687       1.701
route_type                      -0.0480      0.003    -15.161

In [95]:
X = delay_span_dummy
y = data[y_string]

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(y,X)

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.192751
         Iterations: 35
         Function evaluations: 37
         Gradient evaluations: 37


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                     ZeroInflatedPoisson Regression Results                     
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:              ZeroInflatedPoisson   Df Residuals:                  2564854
Method:                             MLE   Df Model:                            6
Date:                  Fri, 13 Sep 2024   Pseudo R-squ.:                -0.01234
Time:                          18:26:17   Log-Likelihood:            -4.9438e+05
converged:                        False   LL-Null:                   -4.8835e+05
Covariance Type:              nonrobust   LLR p-value:                     1.000
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
inflate_const        2.5443      0.003    765.310      0.000       2.538       2.551
delay_span_0-5      -0.7178      0.006   -115.627      0.000      -0.730      -0.706
delay_span_1

In [96]:
X = time_span_dummy
y = data[y_string]

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(y,X)

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.193352
         Iterations: 35
         Function evaluations: 37
         Gradient evaluations: 37


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                     ZeroInflatedPoisson Regression Results                     
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:              ZeroInflatedPoisson   Df Residuals:                  2564854
Method:                             MLE   Df Model:                            6
Date:                  Fri, 13 Sep 2024   Pseudo R-squ.:                -0.01549
Time:                          18:27:28   Log-Likelihood:            -4.9592e+05
converged:                        False   LL-Null:                   -4.8835e+05
Covariance Type:              nonrobust   LLR p-value:                     1.000
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
inflate_const               2.5534      0.003    768.491      0.000       2.547       2.560
time_span_afternoon        -0.1272      0.010    -12.689      0.000      -0.

In [97]:
X = weekday_dummy
y = data[y_string]

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(y,X)

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.191300
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                     ZeroInflatedPoisson Regression Results                     
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:              ZeroInflatedPoisson   Df Residuals:                  2564855
Method:                             MLE   Df Model:                            5
Date:                  Fri, 13 Sep 2024   Pseudo R-squ.:               -0.004715
Time:                          18:28:35   Log-Likelihood:            -4.9066e+05
converged:                        False   LL-Null:                   -4.8835e+05
Covariance Type:              nonrobust   LLR p-value:                     1.000
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
inflate_const         2.3025      0.003    679.366      0.000       2.296       2.309
weekday_Monday       -0.6759      0.008    -84.735      0.000      -0.692      -0.660
weekday_

In [98]:
X = selected_features
y = data[y_string]

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(y,X.astype(float))

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1331: RuntimeWarning: overflow encountered in exp
  return -np.exp(XB) +  endog*XB - gammaln(endog+1)
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1508: RuntimeWarning: overflow encountered in exp
  L = np.exp(np.dot(X,params) + offset + exposure)
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1509: RuntimeWarning: overflow encountered in multiply
  return (self.endog - L)[:,None] * X
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1509: RuntimeWarning: invalid value encountered in multiply
  return (self.endog - L)[:,None] * X
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1074: RuntimeWarning: overflow encountered in exp
  return np.exp(linpred)
/

         Current function value: 0.220646
         Iterations: 0
         Function evaluations: 13
         Gradient evaluations: 2


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                     ZeroInflatedPoisson Regression Results                     
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:              ZeroInflatedPoisson   Df Residuals:                  2564851
Method:                             MLE   Df Model:                            9
Date:                  Fri, 13 Sep 2024   Pseudo R-squ.:                 -0.1588
Time:                          18:29:19   Log-Likelihood:            -5.6593e+05
converged:                        False   LL-Null:                   -4.8835e+05
Covariance Type:              nonrobust   LLR p-value:                     1.000
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
inflate_const           0.1000      0.006     16.773      0.000       0.088       0.112
stop_id                -0.0021    9.2e-06   -230.979      0.000      -0.002      -0.002


In [99]:
# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string], data[['delay_category']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.195057
         Iterations: 13
         Function evaluations: 15
         Gradient evaluations: 15
                     ZeroInflatedPoisson Regression Results                     
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:              ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                             MLE   Df Model:                            0
Date:                  Fri, 13 Sep 2024   Pseudo R-squ.:                -0.02445
Time:                          18:30:04   Log-Likelihood:            -5.0029e+05
converged:                         True   LL-Null:                   -4.8835e+05
Covariance Type:              nonrobust   LLR p-value:                       nan
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
inflate_const 

In [100]:

# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string], data[['cancelled_trip']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.193944
         Iterations: 14
         Function evaluations: 16
         Gradient evaluations: 16
                     ZeroInflatedPoisson Regression Results                     
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:              ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                             MLE   Df Model:                            0
Date:                  Fri, 13 Sep 2024   Pseudo R-squ.:                -0.01860
Time:                          18:30:50   Log-Likelihood:            -4.9744e+05
converged:                         True   LL-Null:                   -4.8835e+05
Covariance Type:              nonrobust   LLR p-value:                       nan
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
inflate_const 

In [101]:
# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data['tier_trips_count'],data[['delay']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.098202
         Iterations: 20
         Function evaluations: 22
         Gradient evaluations: 22
                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:        tier_trips_count   No. Observations:              2564861
Model:             ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                            MLE   Df Model:                            0
Date:                 Fri, 13 Sep 2024   Pseudo R-squ.:                -0.02853
Time:                         18:31:41   Log-Likelihood:            -2.5188e+05
converged:                        True   LL-Null:                   -2.4489e+05
Covariance Type:             nonrobust   LLR p-value:                       nan
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
inflate_const     3.3313

In [102]:
# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string],data[['no_delay']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.195011
         Iterations: 16
         Function evaluations: 18
         Gradient evaluations: 18
                     ZeroInflatedPoisson Regression Results                     
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:              ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                             MLE   Df Model:                            0
Date:                  Fri, 13 Sep 2024   Pseudo R-squ.:                -0.02421
Time:                          18:32:28   Log-Likelihood:            -5.0018e+05
converged:                         True   LL-Null:                   -4.8835e+05
Covariance Type:              nonrobust   LLR p-value:                       nan
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
inflate_const   

In [103]:
# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string],data[['nan_delay']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.194015
         Iterations: 14
         Function evaluations: 16
         Gradient evaluations: 16
                     ZeroInflatedPoisson Regression Results                     
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:              ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                             MLE   Df Model:                            0
Date:                  Fri, 13 Sep 2024   Pseudo R-squ.:                -0.01898
Time:                          18:33:15   Log-Likelihood:            -4.9762e+05
converged:                         True   LL-Null:                   -4.8835e+05
Covariance Type:              nonrobust   LLR p-value:                       nan
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
inflate_const   

In [104]:
# Create the Poisson regression model
model = sm.ZeroInflatedPoisson(data[y_string],data[['nan_delay']])

# Fit the model
results = model.fit()

# Print the summary of the model
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.194015
         Iterations: 14
         Function evaluations: 16
         Gradient evaluations: 16
                     ZeroInflatedPoisson Regression Results                     
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:              ZeroInflatedPoisson   Df Residuals:                  2564860
Method:                             MLE   Df Model:                            0
Date:                  Fri, 13 Sep 2024   Pseudo R-squ.:                -0.01898
Time:                          18:34:01   Log-Likelihood:            -4.9762e+05
converged:                         True   LL-Null:                   -4.8835e+05
Covariance Type:              nonrobust   LLR p-value:                       nan
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
inflate_const   

# Negative Binomial Regression

#### Best Models

In [117]:
X = selected_features
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X.astype(float), family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564850
Model Family:          NegativeBinomial   Df Model:                           10
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -4.8284e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   6.6397e+05
Time:                          19:00:37   Pearson chi2:                 3.86e+06
No. Iterations:                       7   Pseudo R-squ. (CS):            0.01711
Covariance Type:              nonrobust                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const         

In [113]:
X = data_dummy.drop(columns=[y_string, 'departure_delay_float'])
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X.astype(float), family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564829
Model Family:          NegativeBinomial   Df Model:                           31
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -4.8060e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   6.5950e+05
Time:                          19:00:14   Pearson chi2:                 3.84e+06
No. Iterations:                     100   Pseudo R-squ. (CS):            0.01882
Covariance Type:              nonrobust                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------

### Other

In [105]:
X = data[['departure_delay_float_new']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564859
Model Family:          NegativeBinomial   Df Model:                            1
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.0460e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   7.0749e+05
Time:                          18:34:03   Pearson chi2:                 3.69e+06
No. Iterations:                       6   Pseudo R-squ. (CS):          0.0002866
Covariance Type:              nonrobust                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
co

In [106]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['cancelled_trip']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564859
Model Family:          NegativeBinomial   Df Model:                            1
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.0496e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   7.0821e+05
Time:                          18:34:06   Pearson chi2:                 3.69e+06
No. Iterations:                       6   Pseudo R-squ. (CS):          6.524e-06
Covariance Type:              nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -3.013

In [107]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['delay_span_int']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564859
Model Family:          NegativeBinomial   Df Model:                            1
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.0309e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   7.0447e+05
Time:                          18:34:08   Pearson chi2:                 3.65e+06
No. Iterations:                       6   Pseudo R-squ. (CS):           0.001463
Covariance Type:              nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -2.900

In [108]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['delay_category']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564859
Model Family:          NegativeBinomial   Df Model:                            1
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.0447e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   7.0724e+05
Time:                          18:34:10   Pearson chi2:                 3.67e+06
No. Iterations:                       6   Pseudo R-squ. (CS):          0.0003848
Covariance Type:              nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -3.051

In [109]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['delay']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564859
Model Family:          NegativeBinomial   Df Model:                            1
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.0217e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   7.0263e+05
Time:                          18:34:13   Pearson chi2:                 3.60e+06
No. Iterations:                       6   Pseudo R-squ. (CS):           0.002179
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.8728      0.003

In [110]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['no_delay']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564859
Model Family:          NegativeBinomial   Df Model:                            1
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.0199e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   7.0227e+05
Time:                          18:34:15   Pearson chi2:                 3.63e+06
No. Iterations:                       6   Pseudo R-squ. (CS):           0.002318
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1573      0.004

In [111]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['nan_delay']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564859
Model Family:          NegativeBinomial   Df Model:                            1
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.0496e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   7.0822e+05
Time:                          18:34:17   Pearson chi2:                 3.69e+06
No. Iterations:                       6   Pseudo R-squ. (CS):          4.161e-06
Covariance Type:              nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0310      0.004

In [114]:
X = delay_span_dummy
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X.astype(float), family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564853
Model Family:          NegativeBinomial   Df Model:                            7
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.0287e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   7.0404e+05
Time:                          19:00:20   Pearson chi2:                 3.63e+06
No. Iterations:                       7   Pseudo R-squ. (CS):           0.001630
Covariance Type:              nonrobust                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               

In [115]:
X = time_span_dummy
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X.astype(float), family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564853
Model Family:          NegativeBinomial   Df Model:                            7
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.0260e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   7.0349e+05
Time:                          19:00:24   Pearson chi2:                 3.66e+06
No. Iterations:                       6   Pseudo R-squ. (CS):           0.001846
Covariance Type:              nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const 

In [116]:
X = weekday_dummy
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the negative binomial regression model
model = sm.GLM(y, X.astype(float), family=sm.families.NegativeBinomial())
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                  Generalized Linear Model Regression Results                   
Dep. Variable:     nextbike_trips_count   No. Observations:              2564861
Model:                              GLM   Df Residuals:                  2564854
Model Family:          NegativeBinomial   Df Model:                            6
Link Function:                      Log   Scale:                          1.0000
Method:                            IRLS   Log-Likelihood:            -5.0386e+05
Date:                  Fri, 13 Sep 2024   Deviance:                   7.0601e+05
Time:                          19:00:28   Pearson chi2:                 3.67e+06
No. Iterations:                       6   Pseudo R-squ. (CS):          0.0008640
Covariance Type:              nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             

# Zero Inflated Negative Binomial

In [118]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['departure_delay_float_new']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X)

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.189299
         Iterations: 35
         Function evaluations: 38
         Gradient evaluations: 38


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:              nextbike_trips_count   No. Observations:              2564861
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                  2564859
Method:                                      MLE   Df Model:                            1
Date:                           Fri, 13 Sep 2024   Pseudo R-squ.:               0.0003048
Time:                                   19:09:06   Log-Likelihood:            -4.8553e+05
converged:                                 False   LL-Null:                   -4.8567e+05
Covariance Type:                       nonrobust   LLR p-value:                 2.327e-66
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
inflate_const                -0.0393      0.074     -0.529      0.597      -0.185       0.10

In [119]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['delay_span_int']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X)

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.188831
         Iterations: 30
         Function evaluations: 33
         Gradient evaluations: 33
                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:              nextbike_trips_count   No. Observations:              2564861
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                  2564859
Method:                                      MLE   Df Model:                            1
Date:                           Fri, 13 Sep 2024   Pseudo R-squ.:                0.002778
Time:                                   19:18:37   Log-Likelihood:            -4.8432e+05
converged:                                  True   LL-Null:                   -4.8567e+05
Covariance Type:                       nonrobust   LLR p-value:                     0.000
                     coef    std err          z      P>|z|      [0.025      0.975]
-------------------------

In [120]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['delay_category']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X)

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.189216
         Iterations: 30
         Function evaluations: 32
         Gradient evaluations: 32
                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:              nextbike_trips_count   No. Observations:              2564861
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                  2564859
Method:                                      MLE   Df Model:                            1
Date:                           Fri, 13 Sep 2024   Pseudo R-squ.:               0.0007438
Time:                                   19:25:02   Log-Likelihood:            -4.8531e+05
converged:                                  True   LL-Null:                   -4.8567e+05
Covariance Type:                       nonrobust   LLR p-value:                3.922e-159
                     coef    std err          z      P>|z|      [0.025      0.975]
-------------------------

In [121]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['cancelled_trip']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X)

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.189357
         Iterations: 31
         Function evaluations: 34
         Gradient evaluations: 34
                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:              nextbike_trips_count   No. Observations:              2564861
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                  2564859
Method:                                      MLE   Df Model:                            1
Date:                           Fri, 13 Sep 2024   Pseudo R-squ.:              -1.394e-07
Time:                                   19:32:22   Log-Likelihood:            -4.8567e+05
converged:                                  True   LL-Null:                   -4.8567e+05
Covariance Type:                       nonrobust   LLR p-value:                     1.000
                     coef    std err          z      P>|z|      [0.025      0.975]
-------------------------

In [122]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['delay']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X)

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.188557
         Iterations: 30
         Function evaluations: 33
         Gradient evaluations: 33
                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:              nextbike_trips_count   No. Observations:              2564861
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                  2564859
Method:                                      MLE   Df Model:                            1
Date:                           Fri, 13 Sep 2024   Pseudo R-squ.:                0.004222
Time:                                   19:37:24   Log-Likelihood:            -4.8362e+05
converged:                                  True   LL-Null:                   -4.8567e+05
Covariance Type:                       nonrobust   LLR p-value:                     0.000
                    coef    std err          z      P>|z|      [0.025      0.975]
--------------------------

In [123]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['no_delay']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X)

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.188535
         Iterations: 30
         Function evaluations: 33
         Gradient evaluations: 33
                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:              nextbike_trips_count   No. Observations:              2564861
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                  2564859
Method:                                      MLE   Df Model:                            1
Date:                           Fri, 13 Sep 2024   Pseudo R-squ.:                0.004339
Time:                                   19:45:48   Log-Likelihood:            -4.8357e+05
converged:                                  True   LL-Null:                   -4.8567e+05
Covariance Type:                       nonrobust   LLR p-value:                     0.000
                    coef    std err          z      P>|z|      [0.025      0.975]
--------------------------

In [124]:
# Define the independent variables (X) and the dependent variable (y)
X = data[['nan_delay']]
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X)

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.189358
         Iterations: 29
         Function evaluations: 33
         Gradient evaluations: 33
                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:              nextbike_trips_count   No. Observations:              2564861
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                  2564859
Method:                                      MLE   Df Model:                            1
Date:                           Fri, 13 Sep 2024   Pseudo R-squ.:              -4.627e-06
Time:                                   19:52:23   Log-Likelihood:            -4.8568e+05
converged:                                  True   LL-Null:                   -4.8567e+05
Covariance Type:                       nonrobust   LLR p-value:                     1.000
                    coef    std err          z      P>|z|      [0.025      0.975]
--------------------------

In [125]:
# Define the independent variables (X) and the dependent variable (y)
X = data_dummy.drop(columns=[y_string, 'departure_delay_float'])
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X.astype(float))

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:4329: RuntimeWarning: overflow encountered in exp
  return np.exp(linpred)
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:3937: RuntimeWarning: invalid value encountered in multiply
  a1 * np.log(a1) + y * np.log(mu) -
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:3936: RuntimeWarning: invalid value encountered in subtract
  llf = (gammaln(y + a1) - gammaln(y + 1) - gammaln(a1) +
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:3974: RuntimeWarning: divide by zero enc

         Current function value: nan
         Iterations: 1
         Function evaluations: 112
         Gradient evaluations: 112


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:              nextbike_trips_count   No. Observations:              2564861
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                  2564827
Method:                                      MLE   Df Model:                           33
Date:                           Fri, 13 Sep 2024   Pseudo R-squ.:                     nan
Time:                                   20:24:57   Log-Likelihood:                    nan
converged:                                 False   LL-Null:                   -4.8567e+05
Covariance Type:                       nonrobust   LLR p-value:                       nan
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
inflate_const                   -1.0774        nan        nan        nan         nan  

In [126]:
# Define the independent variables (X) and the dependent variable (y)
X = time_span_dummy
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X.astype(float))

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.188706
         Iterations: 35
         Function evaluations: 39
         Gradient evaluations: 39


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:              nextbike_trips_count   No. Observations:              2564861
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                  2564853
Method:                                      MLE   Df Model:                            7
Date:                           Fri, 13 Sep 2024   Pseudo R-squ.:                0.003435
Time:                                   20:34:42   Log-Likelihood:            -4.8401e+05
converged:                                 False   LL-Null:                   -4.8567e+05
Covariance Type:                       nonrobust   LLR p-value:                     0.000
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
inflate_const              -2.6491      0.194    -13.634      0.000      -3.030      -2.268
cons

In [127]:
# Define the independent variables (X) and the dependent variable (y)
X = delay_span_dummy
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X.astype(float))

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.188759
         Iterations: 30
         Function evaluations: 33
         Gradient evaluations: 33
                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:              nextbike_trips_count   No. Observations:              2564861
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                  2564853
Method:                                      MLE   Df Model:                            7
Date:                           Fri, 13 Sep 2024   Pseudo R-squ.:                0.003157
Time:                                   20:45:02   Log-Likelihood:            -4.8414e+05
converged:                                  True   LL-Null:                   -4.8567e+05
Covariance Type:                       nonrobust   LLR p-value:                     0.000
                       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------

In [128]:
# Define the independent variables (X) and the dependent variable (y)
X = weekday_dummy
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X.astype(float))

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.189042
         Iterations: 35
         Function evaluations: 39
         Gradient evaluations: 39


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:              nextbike_trips_count   No. Observations:              2564861
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                  2564854
Method:                                      MLE   Df Model:                            6
Date:                           Fri, 13 Sep 2024   Pseudo R-squ.:                0.001663
Time:                                   20:54:01   Log-Likelihood:            -4.8487e+05
converged:                                 False   LL-Null:                   -4.8567e+05
Covariance Type:                       nonrobust   LLR p-value:                     0.000
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
inflate_const        -2.7008      0.211    -12.776      0.000      -3.115      -2.286
const                -

In [129]:
# Define the independent variables (X) and the dependent variable (y)
X = selected_features
y = data[y_string]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create the zero-inflated negative binomial regression model
model = sm.ZeroInflatedNegativeBinomialP(y, X.astype(float))

# Fit the model
result = model.fit()

# Print the summary of the model
print(result.summary())

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:4329: RuntimeWarning: overflow encountered in exp
  return np.exp(linpred)
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:3937: RuntimeWarning: invalid value encountered in multiply
  a1 * np.log(a1) + y * np.log(mu) -
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:3936: RuntimeWarning: invalid value encountered in subtract
  llf = (gammaln(y + a1) - gammaln(y + 1) - gammaln(a1) +
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:3974: RuntimeWarning: divide by zero enc

         Current function value: nan
         Iterations: 1
         Function evaluations: 112
         Gradient evaluations: 112


/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:              nextbike_trips_count   No. Observations:              2564861
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                  2564850
Method:                                      MLE   Df Model:                           10
Date:                           Fri, 13 Sep 2024   Pseudo R-squ.:                     nan
Time:                                   21:08:01   Log-Likelihood:                    nan
converged:                                 False   LL-Null:                   -4.8567e+05
Covariance Type:                       nonrobust   LLR p-value:                       nan
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
inflate_const          -0.8432        nan        nan        nan         nan         nan
const           

In [130]:
# Zero inflated in thesis erklären

# Multilinear Regression 

### Best Model

In [139]:
# ALL

# Create a copy of the delay dataframe with the selected columns
delay_regression = data[['nextbike_trips_count', 'departure_delay_float', 'current_temp', 'current_precipitation_volume', 'service', 'weekday', 'delay_span', 'time_span', 'weekend', 'delay_category', 'cancelled_trip', 'delay', 'no_delay', 'nan_delay']].copy()

# Fit the multiple linear regression model
model = smf.ols(formula='nextbike_trips_count ~ departure_delay_float + current_temp + current_precipitation_volume + service + weekday + delay_span + time_span + weekend + delay_category + cancelled_trip + delay + no_delay + nan_delay', data=delay_regression).fit()

# Print the summary of the model
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.016
Model:                              OLS   Adj. R-squared:                  0.016
Method:                   Least Squares   F-statistic:                     927.0
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                          22:33:12   Log-Likelihood:            -1.4380e+05
No. Observations:               1522668   AIC:                         2.877e+05
Df Residuals:                   1522641   BIC:                         2.880e+05
Df Model:                            26                                         
Covariance Type:              nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------

### Other

In [131]:
# DELAY FEATURES ONLY

# Create a copy of the delay dataframe with the selected columns
delay_regression = data[[y_string, 'departure_delay_float_new', 'delay', 'no_delay', 'nan_delay', 'delay_span', 'cancelled_trip']].copy()

# Fit the multiple linear regression model
model = smf.ols(formula='nextbike_trips_count ~ departure_delay_float_new + delay + no_delay + nan_delay + delay_span + cancelled_trip', data=delay_regression).fit()

# Print the summary of the model
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.005
Model:                              OLS   Adj. R-squared:                  0.005
Method:                   Least Squares   F-statistic:                     1258.
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                          21:08:12   Log-Likelihood:            -2.8343e+05
No. Observations:               2564861   AIC:                         5.669e+05
Df Residuals:                   2564849   BIC:                         5.670e+05
Df Model:                            11                                         
Covariance Type:              nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
In

In [134]:
# DELAY FEATURES ONLY

# Create a copy of the delay dataframe with the selected columns
delay_regression = data[[y_string, 'departure_delay_float', 'delay', 'no_delay', 'nan_delay', 'delay_span', 'cancelled_trip']].copy()

# Fit the multiple linear regression model
model = smf.ols(formula='nextbike_trips_count ~ departure_delay_float + delay + no_delay + nan_delay + delay_span + cancelled_trip', data=delay_regression).fit()

# Print the summary of the model
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.011
Model:                              OLS   Adj. R-squared:                  0.011
Method:                   Least Squares   F-statistic:                     1892.
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                          21:28:19   Log-Likelihood:            -1.4729e+05
No. Observations:               1522668   AIC:                         2.946e+05
Df Residuals:                   1522658   BIC:                         2.947e+05
Df Model:                             9                                         
Covariance Type:              nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept 

/home/students/miniconda3/envs/Andrej/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


In [135]:
# TIME FEATURES

# Create a copy of the delay dataframe with the selected columns
delay_regression = data[['nextbike_trips_count', 'departure_delay_float_new', 'departure_delay_float', 'delay', 'no_delay', 'nan_delay', 'weekday', 'weekend', 'time_span']].copy()

# Fit the multiple linear regression model
model = smf.ols(formula='nextbike_trips_count ~ departure_delay_float_new + delay + no_delay + nan_delay + time_span + weekday + weekend', data=delay_regression).fit()

# Print the summary of the model
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.005
Model:                              OLS   Adj. R-squared:                  0.005
Method:                   Least Squares   F-statistic:                     804.7
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                          21:28:36   Log-Likelihood:            -2.8391e+05
No. Observations:               2564861   AIC:                         5.679e+05
Df Residuals:                   2564844   BIC:                         5.681e+05
Df Model:                            16                                         
Covariance Type:              nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------


In [136]:
# TIME FEATURES

# Create a copy of the delay dataframe with the selected columns
delay_regression = data[['nextbike_trips_count', 'departure_delay_float_new', 'departure_delay_float', 'delay', 'no_delay', 'nan_delay', 'weekday', 'weekend', 'time_span']].copy()

# Fit the multiple linear regression model
model = smf.ols(formula='nextbike_trips_count ~ departure_delay_float_new + delay + no_delay + nan_delay + time_span + weekday + weekend', data=delay_regression).fit()

# Print the summary of the model
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.005
Model:                              OLS   Adj. R-squared:                  0.005
Method:                   Least Squares   F-statistic:                     804.7
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                          21:28:53   Log-Likelihood:            -2.8391e+05
No. Observations:               2564861   AIC:                         5.679e+05
Df Residuals:                   2564844   BIC:                         5.681e+05
Df Model:                            16                                         
Covariance Type:              nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------


In [138]:
# ALL

# Create a copy of the delay dataframe with the selected columns
delay_regression = data[['nextbike_trips_count', 'departure_delay_float_new', 'current_temp', 'current_precipitation_volume', 'service', 'weekday', 'delay_span', 'time_span', 'weekend', 'delay_category', 'cancelled_trip', 'delay', 'no_delay', 'nan_delay']].copy()

# Fit the multiple linear regression model
model = smf.ols(formula='nextbike_trips_count ~ departure_delay_float_new + current_temp + current_precipitation_volume + service + weekday + delay_span + time_span + weekend + delay_category + cancelled_trip + delay + no_delay + nan_delay', data=delay_regression).fit()

# Print the summary of the model
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     nextbike_trips_count   R-squared:                       0.009
Model:                              OLS   Adj. R-squared:                  0.009
Method:                   Least Squares   F-statistic:                     801.1
Date:                  Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                          22:32:51   Log-Likelihood:            -2.7917e+05
No. Observations:               2564861   AIC:                         5.584e+05
Df Residuals:                   2564832   BIC:                         5.588e+05
Df Model:                            28                                         
Covariance Type:              nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------